<a href="https://colab.research.google.com/github/SevilAnna/GraduationProject/blob/Beyza/EMU403_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>EMÜ 403 - Graduation Project</h1>

---


<font color='red'> 🔴🔴 Kızlar 🔴🔴</font>      

Kod üzerinde çalışacaksanız:
- Öncelikle kodun en son versiyonunun <a href="https://github.com/SevilAnna/GraduationProject">Github</a>'da olduğundan emin olun. 
- Drive'daki "Düzenlenmiş_Dişli_Parçaları" Excelini kendi bilgisayarınıza indirin. 
- Veriyi çekmek için aşağıdaki en az ilk 3 kutudaki kodu çalıştırın.
- Github'a kaydederken "main branch"e değil, başka bir yan branch'e kaydedin.           

## Importing Libraries & the Excel File

### Importing libraries for the project

In [1]:
import pandas as pd
import numpy as np
from numpy.random import default_rng
rng = default_rng()
import random
import copy
#!pip install geneticalgorithm
#import geneticalgorithm as ga
#from geneticalgorithm import geneticalgorithm as ga
print("done")

done


### Importing the Excel file

In [2]:
from google.colab import files
uploaded = files.upload()

Saving Düzenlenmiş_Dişli_Parçaları.xlsx to Düzenlenmiş_Dişli_Parçaları.xlsx


In [3]:
import io
df = pd.read_excel(io.BytesIO(uploaded["Düzenlenmiş_Dişli_Parçaları.xlsx"]), sheet_name='Anka', header=0)
del df['SÜRE (11 parça için)']
del df['1 tanesi için kaç tane gerek']
del df['PROSES']
del df['Process time']
pr_num = list(range(1,len(df)+1))
df["Process #"] = pr_num
df = df.rename(columns={'Makine':'Machine', 'İşlem':'Process', 'SETUP':'Setup', 
                    'SÜRE (1 parça için)':'Processing time', 'PARÇA':'Piece'})
df = df.reindex(columns=['Piece','Process #','Process','Machine','Setup','Processing time'])
df

,Piece,Process #,Process,Machine,Setup,Processing time
0,UM121T900201200 GEAR ENGINE Z35,1,Pre-Hardening Öncesi Torna,D13,0.0,1.000000
1,UM121T900201200 GEAR ENGINE Z35,2,Yıkama,D33,0.0,0.090909
2,UM121T900201200 GEAR ENGINE Z35,3,Ön Sertleştirme (Isıl İşlem),D31,0.0,0.545455
3,UM121T900201200 GEAR ENGINE Z35,4,Yıkama,D33,0.0,0.090909
4,UM121T900201200 GEAR ENGINE Z35,5,Temperleme (Isıl İşlem),D32,0.0,0.454545
...,...,...,...,...,...,...
315,UM121T900204300 GEAR ALTERNATOR Z13 -2,316,Yıkama,D12,0.0,0.181818
316,UM121T900204300 GEAR ALTERNATOR Z13 -2,317,Balans,D69,0.0,1.000000
317,UM121T900204300 GEAR ALTERNATOR Z13 -2,318,MPI,D65,0.0,0.454545
318,UM121T900204300 GEAR ALTERNATOR Z13 -2,319,Yıkama,D12,0.0,0.090909


## Create Chromosomes

In [4]:
m_list = np.unique(np.array(df["Machine"]))                         # Each machine needed
m_num_list = [1, 2, 1, 4, 3, 2, 3, 3, 3, 2, 2, 1, 1, 1, 2, 1, 1]    # How many there're of each machine (done manually now; to automate later)
print(len(m_list),m_list)
print(len(m_num_list),m_num_list)

17 ['662' 'D11' 'D12' 'D13' 'D15' 'D17' 'D27' 'D31' 'D32' 'D33' 'D41' 'D43'
 'D44' 'D47' 'D65' 'D69' 'Nital Etche, Hydrojen Embr.']
17 [1, 2, 1, 4, 3, 2, 3, 3, 3, 2, 2, 1, 1, 1, 2, 1, 1]


In [5]:
# create a dictionary for each machine (including duplicates) & their processes
all_machines = {}
n = 0
for m in m_list:
  m_num = m_num_list[n]
  if m_num > 0:
    all_machines[m] = []
    # Adding processes to the machines (but not to duplicates)
    for r in range(len(df["Machine"])):
      if m == df.iloc[r]["Machine"]:
        all_machines[m].append(r+1)
  # add duplicate machines without the processes
  
  #bu boşları düşünelim(23.05.2022 tarihinde bunları commande ben aldım-beyzza)
  #if m_num > 1:
  #  m1 = m+"-1"
  #  all_machines[m1] = all_machines[m]
  #  all_machines.pop(m)
  #  if m_num > 1:
  #    m2 = m+"-2"
  #    all_machines[m2] = []
  #  if m_num > 2:
  #    m3 = m+"-3"
  #    all_machines[m3] = []
  #  if m_num > 3:
  #    m4 = m+"-4"
  #    all_machines[m4] = []   # later add more if there're 4+ of a certain machine ---> or just write a function for this 
  # n += 1
   
for k in all_machines:
  print(k,"\t",all_machines[k])

print(len(all_machines))

662 	 [26, 32, 58, 64, 90, 96, 122, 128, 154, 160, 186, 192, 218, 224, 250, 256, 282, 288, 314, 320]
D11 	 [11, 23, 43, 55, 75, 87, 107, 119, 139, 151, 171, 183, 203, 215, 235, 247, 267, 279, 299, 311]
D12 	 [13, 24, 28, 31, 45, 56, 60, 63, 77, 88, 92, 95, 109, 120, 124, 127, 141, 152, 156, 159, 173, 184, 188, 191, 205, 216, 220, 223, 237, 248, 252, 255, 269, 280, 284, 287, 301, 312, 316, 319]
D13 	 [1, 7, 33, 39, 65, 71, 97, 103, 129, 135, 161, 167, 193, 199, 225, 231, 257, 263, 289, 295]
D15 	 [14, 20, 25, 46, 52, 57, 78, 84, 89, 110, 116, 121, 142, 148, 153, 174, 180, 185, 206, 212, 217, 238, 244, 249, 270, 276, 281, 302, 308, 313]
D17 	 [10, 22, 42, 54, 74, 86, 106, 118, 138, 150, 170, 182, 202, 214, 234, 246, 266, 278, 298, 310]
D27 	 [21, 53, 85, 117, 149, 181, 213, 245, 277, 309]
D31 	 [3, 15, 35, 47, 67, 79, 99, 111, 131, 143, 163, 175, 195, 207, 227, 239, 259, 271, 291, 303]
D32 	 [5, 18, 37, 50, 69, 82, 101, 114, 133, 146, 165, 178, 197, 210, 229, 242, 261, 274, 293, 306]
D33

In [6]:
machines = copy.deepcopy(all_machines)

# initialize genetic algorithm where each gene is a machine's schedule
num_genes = len(machines)
num_chro = 8
popu_size = (num_chro,num_genes)
popu = []


for i in range(num_chro):
  popu.append([machines])   #popu.append([machines])

# shuffle the order of processes in each machine/gene except for the first chromosome, and print results
new_popu = []
for i in range(len(popu)):
  new_popu.append(0,)    #new_popu.append([0])

first_one = True
for i in range(len(popu)):
  if first_one == False:
    for j in popu[i]:
      for k in j:
        temp_l = random.sample(copy.deepcopy(list(j[k])),len(j[k]))
        j[k] = copy.deepcopy(temp_l)
  first_one = False
  new_popu[i] = copy.deepcopy(popu[i])

for c in new_popu:
  print(c)

[{'662': [26, 32, 58, 64, 90, 96, 122, 128, 154, 160, 186, 192, 218, 224, 250, 256, 282, 288, 314, 320], 'D11': [11, 23, 43, 55, 75, 87, 107, 119, 139, 151, 171, 183, 203, 215, 235, 247, 267, 279, 299, 311], 'D12': [13, 24, 28, 31, 45, 56, 60, 63, 77, 88, 92, 95, 109, 120, 124, 127, 141, 152, 156, 159, 173, 184, 188, 191, 205, 216, 220, 223, 237, 248, 252, 255, 269, 280, 284, 287, 301, 312, 316, 319], 'D13': [1, 7, 33, 39, 65, 71, 97, 103, 129, 135, 161, 167, 193, 199, 225, 231, 257, 263, 289, 295], 'D15': [14, 20, 25, 46, 52, 57, 78, 84, 89, 110, 116, 121, 142, 148, 153, 174, 180, 185, 206, 212, 217, 238, 244, 249, 270, 276, 281, 302, 308, 313], 'D17': [10, 22, 42, 54, 74, 86, 106, 118, 138, 150, 170, 182, 202, 214, 234, 246, 266, 278, 298, 310], 'D27': [21, 53, 85, 117, 149, 181, 213, 245, 277, 309], 'D31': [3, 15, 35, 47, 67, 79, 99, 111, 131, 143, 163, 175, 195, 207, 227, 239, 259, 271, 291, 303], 'D32': [5, 18, 37, 50, 69, 82, 101, 114, 133, 146, 165, 178, 197, 210, 229, 242, 261,

note: might need to remove the outer list from chromosomes

 .

## Fitness function

### Make lists for the processes in each piece

In [7]:
a_list = np.unique(np.array(df["Piece"]))
num = df["Piece"].nunique(a_list[0])
u_num = [0]*num
for i in range(9):
  for item in df["Piece"]:
    if item == a_list[i]: 
      u_num[i] += 1
  #print(u_num[i],"processes for piece",a_list[i])

In [8]:
# process numbers in each piece

processes = []
for i in range(num):
  processes.append([])
#print(processes)

j = 0
k = 0
total = 0
for i in range(len(u_num)):
  total += u_num[i]
  while j < total:
    if j == total:
      k += 1
    #print(i,j,k,total,processes)
    #print(processes[i])
    processes[i].append(j+1)
    j += 1
  #print(total)

for i in processes:
  print(len(i),i)

32 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]
32 [33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64]
64 [65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128]
32 [129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160]
32 [161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192]
32 [193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217,

### Calculating times for each process

In [9]:
# setup, processing & total times for each process (from 1 to 320)

pr_times = list(df["Processing time"])
stp_times = list(df["Setup"])
t_times = []
for i in range(len(pr_times)):
  t_times.append(pr_times[i] + stp_times[i])
#print(pr_times)
#print(stp_times)
#print(t_times)

In [10]:
# function to write setup, processing & total times for each process
def write_times(m_time):
  m_pr_time = copy.deepcopy(m_time)
  m_stp_time = copy.deepcopy(m_time)
  m_tot_time = copy.deepcopy(m_time)
  for m in m_time:
    for i in range(len(m_time[m])):
      for j in range(len(pr_times)):
        if m_time[m][i] == j+1:
          m_pr_time[m][i] = pr_times[j]
          m_stp_time[m][i] = stp_times[j]
          m_tot_time[m][i] = t_times[j]
  return m_pr_time, m_stp_time, m_tot_time

# function to write the start & finishing times for each process
def write_start_finish(m_time,tt_time):
  m_S_time = copy.deepcopy(tt_time)
  m_F_time = copy.deepcopy(tt_time)
  for m in m_time:                    # each machine's list
    for i in range(len(m_time[m])):   # 0 to length of each machine (each process in a machine)
      for j in range(len(t_times)):   # 0 to 319 (every single process)
        if m_time[m][i] == j+1:       # if process in m_time = a process
          if i == 0:
            m_S_time[m][i] = 0        ### start first process at 0 for now
          else:
            m_S_time[m][i] = m_S_time[m][i-1] + tt_time[m][i-1]  # or m_F_time[m][i-1]
          m_F_time[m][i] = m_S_time[m][i] + tt_time[m][i]
  return m_S_time, m_F_time

# function to update start & finish times of each process, taking into account the process sequences
def update_start_finish(m_time,process,u_S_time,u_F_time,tt_time):
  for machine, process_list in m_time.items():
      for p in process_list:
        if p == process:
          if process == 1:
            u_S_time[machine][process_list.index(p)] = 0
          else:
            # find the previous process' finishing time
            for m in all_machines:
              for j in all_machines[m]:
                if j == p-1:
                  prev_m = m
                  prev_F = u_F_time[m][m_time[m].index(p-1)]
            # compare that finishing time to the current start time & pick the max
            u_S_time[machine][process_list.index(p)] = max(prev_F, u_S_time[machine][process_list.index(p)]) 
          u_F_time[machine][process_list.index(p)] = u_S_time[machine][process_list.index(p)] + tt_time[machine][process_list.index(p)]



def calculate_times(chromosome):

  # find process, setup & total duration times
  pr_time, stp_time, t_time = write_times(chromosome)
  ##print("process:",pr_time)
  ##print("setups:",stp_time)
  print("total:",t_time)

  # find start & finish times
  S_time, F_time = write_start_finish(chromosome,t_time)
  print("start  ",S_time)
  print("finish ",F_time)

  # calculate & update the new start & finish times
  new_S_time = copy.deepcopy(S_time)
  new_F_time = copy.deepcopy(F_time)
  for piece in processes:
    for process in piece:
      update_start_finish(chromosome,process,new_S_time,new_F_time,t_time)
  print("\nnew start ",new_S_time)
  print("new finish",new_F_time)
  #print("")
  #for i in new_S_time:
  #  print(i)
  #  print("prs",chromosome[i])
  #  print("t-t",t_time[i])
  #  print("new",new_S_time[i])
  #  print("")

  latest_F_time = 0
  for i in new_F_time:
    #print("asdfgh   ",new_F_time[i])
    max_F_time = max(new_F_time[i],default=0)
    if max_F_time > latest_F_time:
      latest_F_time = max_F_time
      print(latest_F_time)


  #####return t_time, new_S_time, new_F_time

for chr in range(len(new_popu)):
  print(new_popu[chr][0],"\n")
  calculate_times(new_popu[chr][0])
  print("\n---------\n")

{'662': [26, 32, 58, 64, 90, 96, 122, 128, 154, 160, 186, 192, 218, 224, 250, 256, 282, 288, 314, 320], 'D11': [11, 23, 43, 55, 75, 87, 107, 119, 139, 151, 171, 183, 203, 215, 235, 247, 267, 279, 299, 311], 'D12': [13, 24, 28, 31, 45, 56, 60, 63, 77, 88, 92, 95, 109, 120, 124, 127, 141, 152, 156, 159, 173, 184, 188, 191, 205, 216, 220, 223, 237, 248, 252, 255, 269, 280, 284, 287, 301, 312, 316, 319], 'D13': [1, 7, 33, 39, 65, 71, 97, 103, 129, 135, 161, 167, 193, 199, 225, 231, 257, 263, 289, 295], 'D15': [14, 20, 25, 46, 52, 57, 78, 84, 89, 110, 116, 121, 142, 148, 153, 174, 180, 185, 206, 212, 217, 238, 244, 249, 270, 276, 281, 302, 308, 313], 'D17': [10, 22, 42, 54, 74, 86, 106, 118, 138, 150, 170, 182, 202, 214, 234, 246, 266, 278, 298, 310], 'D27': [21, 53, 85, 117, 149, 181, 213, 245, 277, 309], 'D31': [3, 15, 35, 47, 67, 79, 99, 111, 131, 143, 163, 175, 195, 207, 227, 239, 259, 271, 291, 303], 'D32': [5, 18, 37, 50, 69, 82, 101, 114, 133, 146, 165, 178, 197, 210, 229, 242, 261, 

# Crossover


In [11]:
# test son(26.05.2022)

def crossover(parents1, parents2):
  mother = parents1
  father = parents2
  index1 = random.randint(1, len(mth) - 2)
  index2 = random.randint(1, len(mth) - 2)
  if index1 == index2:
    index1 = random.randint(1, len(mth) - 2)
    index2 = random.randint(1, len(mth) - 2)     
  if index1 > index2: index1, index2 = index2, index1   
  child1 = null[:index1] + father[index1:index2] +null[index2:]
  child2 = null[:index1] + mother[index1:index2] + null[index2:]
  #print(index1,index2)
  c=0
  for i in range(len(mth)):
    if child1[i]!=0:
      c=child1[i]
      break
  index_c=0
  for i in range(len(mth)):
    if child1[i]!=0:
      index_c +=1
  index_1=child1.index(c)
  index_2=index_1 + index_c
  VLOOK_1=copy.deepcopy(child1) #eşit olanları silmek için 
  VLOOK_2=copy.deepcopy(child2)
  for i in range(len(mth)):
    j=0
    while j != len(mth):
      if mth[i] != child1[j]:
        VLOOK_1.append(5) # buraya yazdırmam lazımdı o yüzden öylesine işlem atadım.
      else:
        eleman=mth[i]
        VLOOK_1.remove(eleman)
        break
      if j==len(mth)-1 :
        Control_1.append(mth[i])
      j+=1
  for i in range(len(fth)):
    j=0
    while j != len(fth):
      if fth[i] != child2[j]:
        VLOOK_1.append(5) # buraya yazdırmam lazımdı o yüzden öylesine işlem atadım.
      else:
        eleman=fth[i]
        VLOOK_2.remove(eleman)
        break
      if j==len(mth)-1 :
        Control_2.append(fth[i])
      j+=1
  child1 = Control_1[:index_1] + child1[index_1:index_2] +Control_1[index_1:]
  #print("mth",mth)
  child2 = Control_2[:index_1] + child2[index_1:index_2] +Control_2[index_1:]
  return (child1, child2)


Kromozom1={'662': [26, 32, 58, 64, 90, 96, 122, 128, 154, 160, 186, 192, 218, 224, 250, 256, 282, 288, 314, 320], 'D11': [11, 23, 43, 55, 75, 87, 107, 119, 139, 151, 171, 183, 203, 215, 235, 247, 267, 279, 299, 311], 'D12': [13, 24, 28, 31, 45, 56, 60, 63, 77, 88, 92, 95, 109, 120, 124, 127, 141, 152, 156, 159, 173, 184, 188, 191, 205, 216, 220, 223, 237, 248, 252, 255, 269, 280, 284, 287, 301, 312, 316, 319], 'D13': [1, 7, 33, 39, 65, 71, 97, 103, 129, 135, 161, 167, 193, 199, 225, 231, 257, 263, 289, 295], 'D15': [14, 20, 25, 46, 52, 57, 78, 84, 89, 110, 116, 121, 142, 148, 153, 174, 180, 185, 206, 212, 217, 238, 244, 249, 270, 276, 281, 302, 308, 313], 'D17': [10, 22, 42, 54, 74, 86, 106, 118, 138, 150, 170, 182, 202, 214, 234, 246, 266, 278, 298, 310], 'D27': [21, 53, 85, 117, 149, 181, 213, 245, 277, 309], 'D31': [3, 15, 35, 47, 67, 79, 99, 111, 131, 143, 163, 175, 195, 207, 227, 239, 259, 271, 291, 303], 'D32': [5, 18, 37, 50, 69, 82, 101, 114, 133, 146, 165, 178, 197, 210, 229, 242, 261, 274, 293, 306], 'D33': [2, 4, 16, 34, 36, 48, 66, 68, 80, 98, 100, 112, 130, 132, 144, 162, 164, 176, 194, 196, 208, 226, 228, 240, 258, 260, 272, 290, 292, 304], 'D41': [17, 49, 81, 113, 145, 177, 209, 241, 273, 305], 'D43': [9, 41, 73, 105, 137, 169, 201, 233, 265, 297], 'D44': [8, 12, 40, 44, 72, 76, 104, 108, 136, 140, 168, 172, 200, 204, 232, 236, 264, 268, 296, 300], 'D47': [6, 19, 38, 51, 70, 83, 102, 115, 134, 147, 166, 179, 198, 211, 230, 243, 262, 275, 294, 307], 'D65': [30, 62, 94, 126, 158, 190, 222, 254, 286, 318], 'D69': [29, 61, 93, 125, 157, 189, 221, 253, 285, 317], 'Nital Etche, Hydrojen Embr.': [27, 59, 91, 123, 155, 187, 219, 251, 283, 315]}
Kromozom2={'662': [154, 320, 90, 288, 58, 128, 218, 282, 26, 250, 64, 192, 314, 122, 160, 32, 224, 256, 96, 186], 'D11': [279, 311, 107, 119, 203, 139, 55, 75, 183, 23, 151, 171, 235, 247, 87, 267, 43, 11, 299, 215], 'D12': [156, 220, 141, 319, 191, 127, 56, 63, 188, 316, 237, 45, 92, 120, 109, 301, 287, 88, 31, 28, 173, 248, 24, 255, 13, 216, 124, 269, 184, 77, 159, 312, 252, 95, 223, 152, 284, 205, 60, 280], 'D13': [129, 71, 199, 289, 65, 295, 167, 161, 97, 103, 263, 39, 135, 231, 1, 225, 257, 7, 193, 33], 'D15': [238, 153, 244, 84, 174, 142, 180, 25, 116, 302, 110, 249, 185, 270, 20, 57, 217, 89, 206, 212, 313, 46, 308, 276, 78, 148, 281, 121, 52, 14], 'D17': [246, 298, 234, 118, 74, 138, 150, 10, 214, 182, 106, 310, 86, 22, 42, 170, 278, 202, 54, 266], 'D27': [149, 213, 245, 21, 277, 117, 309, 181, 85, 53], 'D31': [3, 259, 303, 163, 239, 111, 271, 67, 131, 291, 15, 35, 175, 207, 99, 227, 143, 47, 195, 79], 'D32': [197, 18, 165, 146, 242, 229, 50, 210, 114, 306, 82, 5, 274, 261, 69, 37, 133, 101, 178, 293], 'D33': [226, 112, 132, 240, 34, 4, 36, 272, 98, 48, 100, 144, 208, 66, 292, 176, 16, 228, 196, 260, 290, 130, 68, 258, 80, 304, 162, 2, 194, 164], 'D41': [305, 113, 81, 49, 17, 145, 177, 209, 273, 241], 'D43': [73, 233, 265, 41, 297, 137, 201, 169, 105, 9], 'D44': [200, 300, 76, 44, 108, 204, 296, 40, 236, 8, 168, 136, 140, 12, 232, 264, 72, 172, 268, 104], 'D47': [147, 211, 307, 115, 275, 243, 134, 294, 262, 19, 51, 102, 230, 38, 70, 198, 166, 6, 179, 83], 'D65': [158, 30, 254, 62, 286, 190, 94, 222, 126, 318], 'D69': [221, 157, 29, 253, 61, 317, 285, 189, 125, 93], 'Nital Etche, Hydrojen Embr.': [251, 27, 91, 315, 187, 123, 155, 59, 283, 219]}

print("Kromozom1",Kromozom1)
print("Kromozom2",Kromozom2)


###############################


popu = []
popu.append(Kromozom1)
popu.append(Kromozom2)
#print(popu)
list1=popu[0] 
list2=popu[1]
#print(list1)

total_chromo_1 = []
total_chromo_2 = []
popp = copy.deepcopy(new_popu)


for i in range(int((len(popp)-2)/2)):
  for m in list1:
    #print(m,list1[m],'\n')
    mth=copy.deepcopy(list1[m]) #mother chorosome's gene for machine m
    fth=copy.deepcopy(list2[m])
    null=[] 
    for j in range (len(mth)):
      null.append(0)    
    Control_1=[]
    Control_2=[]

    mth1,fth1 = crossover(mth,fth) # Durum1
    total_chromo_1.append(mth1)
    total_chromo_2.append(fth1)


first_one = True


len_tc =len(total_chromo_1)

final_new_yavru_popp=[]
final_new_yavru_popp.append(list1)
final_new_yavru_popp.append(list2)

for m in range(int((len(popp)-2)/2)):
  new_yavru_popp= {}
  for j in popp[m_num]:   #chromosome
    for machine in j: 
      new_yavru_popp[str(machine)]= (total_chromo_1[-len_tc])
      len_tc -= 1
  final_new_yavru_popp.append(new_yavru_popp)
  #print("new_yavru_popp=",m+1,new_yavru_popp)

for m in range(int((len(popp)-2)/2)):
  new_yavru_popp= {}
  for j in popp[m_num]:   #chromosome
    for machine in j: 
      new_yavru_popp[str(machine)]= (total_chromo_2[-len_tc])
      len_tc-= 1
  final_new_yavru_popp.append(new_yavru_popp)

for i in final_new_yavru_popp:
  print("new_yavru_popp=",i)


Kromozom1 {'662': [26, 32, 58, 64, 90, 96, 122, 128, 154, 160, 186, 192, 218, 224, 250, 256, 282, 288, 314, 320], 'D11': [11, 23, 43, 55, 75, 87, 107, 119, 139, 151, 171, 183, 203, 215, 235, 247, 267, 279, 299, 311], 'D12': [13, 24, 28, 31, 45, 56, 60, 63, 77, 88, 92, 95, 109, 120, 124, 127, 141, 152, 156, 159, 173, 184, 188, 191, 205, 216, 220, 223, 237, 248, 252, 255, 269, 280, 284, 287, 301, 312, 316, 319], 'D13': [1, 7, 33, 39, 65, 71, 97, 103, 129, 135, 161, 167, 193, 199, 225, 231, 257, 263, 289, 295], 'D15': [14, 20, 25, 46, 52, 57, 78, 84, 89, 110, 116, 121, 142, 148, 153, 174, 180, 185, 206, 212, 217, 238, 244, 249, 270, 276, 281, 302, 308, 313], 'D17': [10, 22, 42, 54, 74, 86, 106, 118, 138, 150, 170, 182, 202, 214, 234, 246, 266, 278, 298, 310], 'D27': [21, 53, 85, 117, 149, 181, 213, 245, 277, 309], 'D31': [3, 15, 35, 47, 67, 79, 99, 111, 131, 143, 163, 175, 195, 207, 227, 239, 259, 271, 291, 303], 'D32': [5, 18, 37, 50, 69, 82, 101, 114, 133, 146, 165, 178, 197, 210, 229, 

newpopudaki chrmozomlara tek tek bak </br>
random sayı ile mutasyn olacak mı diye belirle </br>
olacaksa:</br>
genlere/makine listelerine tek tek bak ve hangisinde olacağına karar ver randon ile </br>
listedeki bir processi rastgele seç ve bir değişkene ata </br>
o precessi liseden sil </br>
değişkeni/processi rastgele başka indexe insert() et

In [12]:
#son hali
new_cont_popp ={}
mutasyon_sayısı = []

# mutation rate
def mutation_rate(key_s,gen,say1,say2):
  new_cont_popp[str(k)]=gen
  #print("1print(new_cont_popp)",new_cont_popp)
  r_mut = 1.0 /len(gen)
  r_mut_range=[1,2,3,4,5]
  n=random.randint(1,100)
  if n in  r_mut_range:
    print("There will be mutation") #bunun çıktısını vermemize gerek yok kontrol amaçlı verdim
    mutasyon_sayısı.append(1)
    n=gen[0][say1]
    m=gen[0][say2]
    gen[0][say1]=m
    gen[0][say2]=n
    new_cont_popp[str(k)]=gen
    #print("2print(new_cont_popp)",new_cont_popp)    
  #else:
  #  print("There will not be mutation")

cont_popp = copy.deepcopy(new_popu)

first_one = True
for j in cont_popp[i]:   #chromosome
  for k in (j.keys()):
    range= []
    list_kromozome =[]
    #print(k,":",j[str(k)],len(j[str(k)]))
    range.append(len(j[str(k)]))
    list_kromozome.append(j[str(k)])
    #print(j[str(k)])
    mut1=random.randint(0,range[0])
    mut2=random.randint(0,range[0])
    if mut1 == mut2:
      while mut1 == mut2:
        mut2=random.randint(0,range[0])
    mutation_rate(k,list_kromozome,mut1,mut2)

print("After mutation part")
print(new_cont_popp)
print("Mutasyon olan gen sayısı:",len(mutasyon_sayısı))

TypeError: ignored

.

    

---
   



### To do:
  - add to finish time calculations:
    - make it so predecession starts from beginning for each piece (eg: 32 --> 33)
    - need to also consider the max of the previous process in the machine           

   
  - check the processes in order across machines & give partial score when testing fitness
  - check the cutoff points of the crossover, the randomness & then generalize it to fit as many chromosomes as needed